## Worksheet 07
***

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse import dok_matrix, coo_matrix
from scipy.sparse.linalg import lsqr

%load_ext autoreload
%autoreload 2

In [ ]:
def constr_X(M, alphas, Np=None):
    """
    Construct design matrix X
    : param M: int 
        Tomogram is an array-like of shape (M, M)
    
    : param alphas: Array-like of shape (n_alphas, )
        List of measurement angles in degrees
        
    : param Np: Default=None
        Optional sensor resolution
    """
    # Define sensor size
    if Np is None:
        Np = int(np.ceil(np.sqrt(2) * M))
        if Np % 2 == 0
            Np += 1
    
    # Define design matrix X
    D_beta = M * M
    D_y = len(alpha) * Np
    
    X = np.empty((D_beta, D_y))
    
    # Loop through all measurement angles
    for i, alpha in enumerate(alphas):
        # Tranform angle to radians and define rotation matrix (instead of rotating sensor matrix)
        alpha_rad = np.radians(alpha)
        rot_matrix = np.array([np.cos(alpha_rad), -np.sin(alpha_rad)],
                              [np.sin(alpha_rad), np.cos(alpha_rad)])
        for y in range(M):
            for x in range(M):
                # Center coordinates
                p = x - (M - 1) / 2
                q = y - (M - 1) / 2
                # Rotate matrix s.t. sensor array and design matrix are aligned for the particular measurement
                p, q = rot_matrix.dot([p, q])
                # Calculate intersection xray with sensor array
                pos = p + (Np - 1) / 2
                
                # Find negihboring sensor elements
                el0 = int(np.floor(pos))
                el1 = int(np.ceil(pos))
                
                # X keeps track on how much pixel (x, y) contributes to the two sensor elements for the particular measurement
                if el0 == el1:
                    X[i * Np + el0, y * M + x] = 1.0
                else:
                    # Interpolation coefficients
                    val0 = el1 - pos
                    val1 = pos - el0
                    
                    # Corner cases
                    if el0 == 0:
                        X[i * Np, y * M + x] = val1
                    elif el0 == Np - 1:
                        X[i * Np + el0, y * M + x] = val0
                    # Interpolation
                    else:
                        X[i * Np + el0, y * M + x] = val0
                        X[i * Np + el1, y * M + x] = val1
    return X